In [1]:
from birdcall.data import *
from birdcall.metrics import *

import pandas as pd

In [2]:
classes = pd.read_pickle('data/classes.pkl')
train_ds = MelspecPoolDataset(pd.read_pickle('data/train_set.pkl'), classes, len_mult=60)
valid_ds = MelspecPoolDataset(pd.read_pickle('data/val_set.pkl'), classes, len_mult=50)

In [3]:
len(train_ds), len(valid_ds)

(15840, 13200)

In [4]:
import torch
import torchvision
from torch import nn
import numpy as np

In [5]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=NUM_WORKERS)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=2*16, shuffle=False, num_workers=NUM_WORKERS)

In [6]:
for b in train_dl: break
b[0].shape, b[1]

(torch.Size([16, 10, 3, 80, 212]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64))

In [7]:
b[0].mean(), b[0].std()

(tensor(0.0540), tensor(0.9483))

In [8]:
def lme_pool(x, alpha=1.0): # log-mean-exp pool
    '''alpha -> approximates maxpool, alpha -> 0 approximates mean pool'''
    T = x.shape[1]
    return 1/alpha * torch.log(1/T * torch.exp(alpha * x).sum(1))

In [9]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(*list(torchvision.models.resnet34(True).children())[:-2])
        self.classifier = nn.Sequential(*[
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=0.5), nn.BatchNorm1d(512),
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=0.5), nn.BatchNorm1d(512),
            nn.Linear(512, len(classes))
        ])
        self.alpha = 1
    
    def forward(self, x):
        bs, im_num, ch, y_dim, x_dim = x.shape
        x = self.cnn(x.view(-1, ch, y_dim, x_dim))
        x = x.mean((2,3))
        x = self.classifier(x)
        x = x.view(bs, im_num, -1)
        x = lme_pool(x, self.alpha)
        return x

In [10]:
model = Model().cuda()

In [11]:
model.alpha = 5
model.load_state_dict(torch.load('models/120_lmepool_0.67.pth'))

<All keys matched successfully>

In [12]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
import time

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), 1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 5)

In [13]:
starting_alpha = 0.05
target_alpha = 15

t0 = time.time()
for epoch in range(300):
    model.alpha = (epoch + 1) / 300 * target_alpha + (1 - (epoch + 1) / 300) * starting_alpha
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        model.train()
        inputs, labels = data[0].cuda(), data[1].cuda()
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if np.isnan(loss.item()): 
            print(f'!!! nan encountered in loss !!! alpha: {model.alpha} epoch: {epoch}\n')
            break
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()


    if epoch % 5 == 4:
        model.eval();
        preds = []
        targs = []

        with torch.no_grad():
            for data in valid_dl:
                inputs, labels = data[0].cuda(), data[1].cuda()
                outputs = model(inputs)
                preds.append(outputs.cpu().detach())
                targs.append(labels.cpu().detach())

            preds = torch.cat(preds)
            targs = torch.cat(targs)

        accuracy = accuracy_score(preds.sigmoid() > 0.5, targs)
        f1 = f1_score(preds.sigmoid() > 0.5, targs, average='micro')
        print(f'[{epoch + 1}, {(time.time() - t0)/60:.1f}] loss: {running_loss / (len(train_dl)-1):.3f}, acc: {accuracy:.3f}, f1: {f1:.3f}')
        running_loss = 0.0

        torch.save(model.state_dict(), f'models/{epoch+1}_lmepool_alpha_{model.alpha}_{round(f1, 2)}.pth')

f1s = []
ts = []

for t in np.linspace(0.4, 1, 61):
    f1s.append(f1_score(preds.sigmoid() > t, targs, average='micro'))
    ts.append(t)

print(f'alpha: {model.alpha}, max f1: {max(f1s)}, best threshold: {ts[np.argmax(f1s)]}')

[5, 24.1] loss: 0.003, acc: 0.553, f1: 0.671
[10, 46.1] loss: 0.003, acc: 0.541, f1: 0.675
[20, 90.8] loss: 0.003, acc: 0.525, f1: 0.665
[30, 134.7] loss: 0.003, acc: 0.549, f1: 0.688
[35, 156.6] loss: 0.003, acc: 0.539, f1: 0.677


NameError: name 'alpha' is not defined

In [17]:
model.eval();
preds = []
targs = []

with torch.no_grad():
    for data in valid_dl:
        inputs, labels = data[0].cuda(), data[1].cuda()
        outputs = model(inputs)
        preds.append(outputs.cpu().detach())
        targs.append(labels.cpu().detach())

    preds = torch.cat(preds)
    targs = torch.cat(targs)

In [18]:
accuracy_score(preds.sigmoid() > 0.5, targs), f1_score(preds.sigmoid() > 0.5, targs, average='micro')

(0.0, 0.0)

In [20]:
torch.__version__

'1.4.0'

In [ ]:
35_lmepool_alpha_1.7941666666666667_0.68.pth

In [19]:
f1s = []
ts = []

for t in np.linspace(0.4, 1, 61):
    f1s.append(f1_score(preds.sigmoid() > t, targs, average='micro'))
    ts.append(t)

print(f'alpha: {model.alpha}, max f1: {max(f1s)}, best threshold: {ts[np.argmax(f1s)]}')

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), f'models/{epoch+1}_lmepool_alpha_{model.alpha}_{round(f1, 2)}.pth')